## Column Type Annotation for Custom Tables with ArcheType

In [1]:
!git clone https://github.com/penfever/archetype/
import os
os.chdir("archetype")

!pip install -r requirements.txt -q

Cloning into 'archetype'...
remote: Enumerating objects: 459, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 459 (delta 182), reused 221 (delta 126), pack-reused 155
Receiving objects: 100% (459/459), 8.98 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (255/255), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 427.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━

IF USING T5:

1. Make sure that your Colab notebook has GPU enabled.
2. In const.py, replace the ARCHETYPE_PATH variable with the absolute path to your ArcheType installation.

IF USING GPT

1. Create an account with OpenAI and use python-dotenv to store your OpenAI API key.
2. In const.py, replace the DOTENV_PATH variable with the absolute path to the directory containing your dotenv with your OpenAI API key.

In [2]:
!cat /content/archetype/src/const.py

EST_CHARS_PER_TOKEN=4
MAX_LEN=2000*EST_CHARS_PER_TOKEN
INTEGER_SET = set(r"0123456789,/\+-.^_()[] :")
BOOLEAN_SET = set(["True", "true", "False", "false", "yes", "Yes", "No", "no"])

ARCHETYPE_PATH = "/home/bf996/archetype"
DOTENV_PATH = "/home/bf996/.env"


In [3]:
!sed 's/\/home\/bf996\//\/content\//g' /content/archetype/src/const.py > /content/archetype/src/const_mod.py
!rm /content/archetype/src/const.py
!mv /content/archetype/src/const_mod.py /content/archetype/src/const.py

## Data

In [4]:
import pandas as pd
from src.predict import ArcheTypePredictor

TEST_FILE_PATH = "./table_samples/Book_5sentidoseditora.pt_September2020_CTA.json"
df = pd.read_json(TEST_FILE_PATH, lines=True)

In [5]:
df.head()

,0,1,2,3
0,Desejo Subtil (eBook),978-972-0-68199-7,336,2013-06-12
1,Desejo Subtil,978-972-0-04396-2,336,2014-04-23
2,Rendida,978-972-0-04429-7,352,2016-05-25
3,Preferida,978-989-745-023-5,448,2016-12-05
4,Confia em mim,978-989-745-028-0,272,2016-06-16


## Model

In [6]:
args = {
            "model_name": "flan-t5-base-zs",
            "custom_labels" : ["text", "number", "id", "place"],
}

arch = ArcheTypePredictor(input_files = [df], user_args = args)
new_df = arch.annotate_columns()

Initializing model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

## Results

In [7]:
new_df.head()

,text,number,number,text
0,Desejo Subtil (eBook),978-972-0-68199-7,336,2013-06-12
1,Desejo Subtil,978-972-0-04396-2,336,2014-04-23
2,Rendida,978-972-0-04429-7,352,2016-05-25
3,Preferida,978-989-745-023-5,448,2016-12-05
4,Confia em mim,978-989-745-028-0,272,2016-06-16
